In [2]:
import pandas as pd

In [ ]:
# 读取Excel文件
file_path = "E:/Test/Inputs/WWTP.xlsx"
df = pd.read_excel(file_path)

In [4]:
# 定义处理函数
def correct_moisture_rate(group):
    # 将超过100的异常值设置为NaN
    group['污泥含水率（%）'] = group['污泥含水率（%）'].apply(lambda x: x if 0 <= x <= 100 else None)
    # 计算有效值的均值
    valid_mean = group['污泥含水率（%）'].mean()
    # 用均值填充空值和异常值
    group['污泥含水率（%）'].fillna(valid_mean, inplace=True)
    # 如果仍有空值（即该污水厂只有空值或异常值），填充默认值80
    group['污泥含水率（%）'].fillna(80, inplace=True)
    return group

In [5]:
# 按“项目编号”分组，并应用处理函数
df = df.groupby('项目编号').apply(correct_moisture_rate)

In [7]:
# 将处理后的数据保存回Excel
output_path = "E:/Test/Outputs/WWTP_corrected.xlsx"
df = pd.read_excel(output_path)
# df.to_excel(output_path, index=False)

In [8]:
# 定义计算含水率为45%时的污泥量的函数
def calculate_new_sludge_amount(row):
    original_amount = row['本月脱水污泥产生量（吨/月）']
    o_land = row['其中,土地利用量（吨）']
    o_build = row['其中,建材利用量（吨）']
    o_inci = row['其中,焚烧利用量（吨）']
    o_lfill = row['其中,卫生填埋量（吨）']
    o_other = row['其中,其他处置量（吨）']
    
    original_wt = row['污泥含水率（%）'] / 100  # 转换为小数
    target_wt = 0.45  # 目标含水率
    
    new_amount = (original_amount * (1 - original_wt)) / (1 - target_wt)
    new_land =  (o_land * (1 - original_wt)) / (1 - target_wt)
    new_build =  (o_build * (1 - original_wt)) / (1 - target_wt)
    new_inci =  (o_inci * (1 - original_wt)) / (1 - target_wt)
    new_lfill =  (o_lfill * (1 - original_wt)) / (1 - target_wt)
    new_other =  (o_other * (1 - original_wt)) / (1 - target_wt)
    
    return pd.Series({'含水率45%时的总污泥处理量（吨/月）': new_amount,
                      '含水率45%时的土地利用量（吨）': new_land,
                      '含水率45%时的建材利用量（吨）': new_build,
                      '含水率45%时的焚烧利用量（吨）': new_inci,
                      '含水率45%时的卫生填埋量（吨）': new_lfill,
                      '含水率45%时的其他处置量（吨）': new_other
                     })

In [10]:
# 计算新的污泥量和土地利用量
df[['含水率45%时的总污泥处理量（吨/月）', '含水率45%时的土地利用量（吨）','含水率45%时的建材利用量（吨）','含水率45%时的焚烧利用量（吨）','含水率45%时的卫生填埋量（吨）','含水率45%时的其他处置量（吨）']] = df.apply(calculate_new_sludge_amount, axis=1)

In [11]:
# 将处理后的数据保存到新的Excel文件
df.to_excel(output_path, index=False)